<a href="https://colab.research.google.com/github/RussAbbott/Wyrdl/blob/main/Wyrdl.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
# This is the start of a longer word list.

wordle_word_list = [
    'aback', 'abase', 'abate', 'abbey', 'abbot', 'abhor', 'abide', 'abled', 'abode', 'abort',
    'about', 'above', 'abuse', 'abyss', 'acorn', 'acrid', 'actor', 'acute', 'adage', 'adapt',
    'adept', 'admin', 'admit', 'adobe', 'adopt', 'adore', 'adorn', 'adult', 'affix', 'afire',
    'afoot', 'afoul', 'after', 'again', 'agape', 'agate', 'agent', 'agile', 'aging', 'aglow',
    'agony', 'agora', 'agree', 'ahead', 'aider', 'aisle', 'alarm', 'album', 'alert', 'algae',
    'alibi', 'alien', 'align', 'alike', 'alive', 'allay', 'alley', 'allot', 'allow', 'alloy',
    'aloft', 'alone', 'along', 'aloof', 'aloud', 'alpha', 'altar', 'alter', 'amass', 'amaze',
    'amber', 'amble', 'amend', 'amiss', 'amity', 'among', 'ample', 'amply', 'amuse', 'angel',
    'anger', 'angle', 'angry', 'angst', 'anime', 'ankle', 'annex', 'annoy', 'annul', 'anode',
    'antic', 'anvil', 'aorta', 'apart', 'aphid', 'aping', 'apnea', 'apple', 'apply', 'apron',
    'aptly', 'arbor', 'ardor', 'arena', 'argue', 'arise', 'armor', 'aroma', 'arose', 'array',
    'arrow', 'arson', 'artsy', 'ascot', 'ashen', 'aside', 'askew', 'assay', 'asset', 'atoll',
    'atone', 'attic', 'audio', 'audit', 'augur', 'aunty', 'avail', 'avert', 'avian', 'avoid',
    'await', 'awake', 'award', 'aware', 'awash', 'awful', 'awoke', 'axial', 'axiom', 'axion',
    'azure']



# wyrdl.py

import contextlib
import pathlib
import random
from string import ascii_letters, ascii_uppercase

from rich.console import Console
from rich.theme import Theme

from typing import List, Tuple, Dict

console = Console(width=80, theme=Theme({"warning": "red on yellow"}))

NUM_LETTERS = 5
NUM_GUESSES = 6
# WORDS_PATH = pathlib.Path(__file__).parent / "wordlist.txt"
WORDS_LIST = wordle_word_list
# BLANK = "_" * NUM_LETTERS

# STYLES map scores to styles. E.g., the score 'g' (for 'green') is mapped to the style "bold black on #00FF00".
STYLES = {'g': "bold black on #00FF00",
          'y': "bold black on #FFFF00",
          '#': "bold white on #000000",
          '_': "dim"
          }

# SCORE assigns an integer to each score so that they can be compared with '>'.
# The ordering is '_#yg'.
SCORE_RANKING: Dict[str, int] = {score: rank for rank, score in enumerate('_#yg')}

verbose = False

def main(verbose=False):
    verbose = verbose

    # Pre-process
    # secret_word = get_random_word(WORDS_PATH.read_text(encoding="utf-8").split("\n"))
    secret_word = get_random_word(WORDS_LIST)
    if verbose:
        print(secret_word)

    # guesses = [BLANK] * NUM_GUESSES -- No longer used

    # guesses is a list of the guesses
    guesses = []
    # styled_guesses is a list of strings reflecting the guesses as styled
    styled_guesses = []

    # letter_status is a dictionary of letters mapped to a (score, style) tuple.
    # For any letter, the score and style are updated to the highest score found.
    letter_status: Dict[str, Tuple[str, str]] = {letter: ('_', styled_letter_and_score(letter, '_')) for letter in ascii_uppercase}

    # Process (main loop)
    with contextlib.suppress(KeyboardInterrupt):
        for idx in range(1, NUM_GUESSES + 1):
            refresh_page(headline=f"Guess {idx}")
            guesses, styled_guesses, letter_status = take_a_guess(secret_word, guesses, styled_guesses, letter_status)
            show_guesses_and_letter_status(styled_guesses, letter_status)
            if guesses[-1] == secret_word:
                break

    # Post-process
    game_over(styled_guesses, secret_word, letter_status, guesses[-1]==secret_word)

def game_over(guesses, word, letter_status, guessed_correctly):
    # refresh_page(headline="Game Over")
    # show_guesses_and_letter_status(guesses, letter_status)

    if guessed_correctly:
        console.print(f"\n[bold black on #ddffdd]Correct, the word is {word}[/]")
    else:
        console.print(f"\n[bold black on #ffdddd]Sorry, the word was {word}[/]")

def get_random_word(word_list):
    if words := [word.upper() for word in word_list
                    if len(word) == NUM_LETTERS and
                       all(letter in ascii_letters for letter in word)]:
        return random.choice(words)
    else:
        console.print(
            f"No words of length {NUM_LETTERS} in the word list",
            style="warning",
        )
        raise SystemExit()

def guess_a_word(previous_guesses) -> str:
    guess = console.input("\nGuess word: ").upper()

    if guess in previous_guesses:
        console.print(f"You've already guessed {guess}.", style="warning")
        return guess_a_word(previous_guesses)

    if len(guess) != NUM_LETTERS:
        console.print(
            f"Your guess must be {NUM_LETTERS} letters.", style="warning"
        )
        return guess_a_word(previous_guesses)

    if any((invalid := letter) not in ascii_letters for letter in guess):
        console.print(
            f"Invalid letter: '{invalid}'. Please use English letters.", style="warning",
        )
        return guess_a_word(previous_guesses)

    return guess

def refresh_page(headline):
    console.clear()
    console.rule(f"[#00CCFF]:mage: {headline} :bulb:[/]\n")

def score_a_guess(secret_word, guess) -> str:
    '''
    Build a list with one char for each guess element.
    g = Letter in Correct position (green);
    y = Letter in Incorrect position (yellow);
    # = Letter not in secret_word. (These are the gray results. Can't use 'g' since we use it for green)

    In the following example: secret_word    = 'hpaay'
                              guess     = 'aaapz'
                              -> score  = 'y#gy#'

    The third letter in both the secret_word and the guess is "a".
        So the third letter in the score must be "g".
        (The matching letters must be determined first.)
    The first letter in the guess, 'a', is associated with the fourth letter in the secret_word, 'a'.
        So the first letter in the score is "y".
    The second letter in the guess, 'a', has nothing in secret_word to be associated with.
        So the second letter in the score is "#".
    The other two letter in the score should be self explanatory.
    '''

    # Generate the score list with the letters in the correct position marked 'g'.
    # All non-green elements are marked '#'. These will be changed to 'y' later if appropriate.
    score = [('g' if sec_ltr == guess_ltr else '#') for sec_ltr, guess_ltr in zip(secret_word, guess)]

    # Generate a list of the "holes," i.e., secret_word letters that do not have correct corresponding guess letters.
    secret_word_holes = [sec_ltr for sec_ltr, score_ltr in zip(secret_word, score) if score_ltr != 'g']

    if verbose: print(f'{secret_word_holes = }')

    # For each available secret_word hole find a matching guess letters. Ignore guess letters that are paired with
    # their secret_word letters. Score the first match as 'y' and don't look further.
    for ltr in secret_word_holes:
        for i in range(len(score)):
            # Have we found a hole-filling letter in guess that is not otherwise committed?
            # If so, mark it 'y'. Since we have now filled this hole, go on to the next hole.
            if guess[i] == ltr and score[i] == '#':
                score[i] = 'y'
                break

    # Return score as a string
    score_string = ''.join(score)
    if verbose:
        print(f'\n\t{score_string = }\n\n')
    return score_string

def show_guesses_and_letter_status(styled_guesses, letter_status):
    for styled_guess in styled_guesses:
        console.print("".join(styled_guess), justify="center")

    console.print("\n" + " ".join([letter_status[letter][1] for letter in letter_status]) + "\n\n", justify="center")

def styled_letter_and_score(letter, score):
    return f"[{STYLES[score]}]{letter}[/]"

def take_a_guess(secret_word: str,
                 guesses: List[str],
                 styled_guesses: List[str],
                 letter_status: Dict[str, Tuple[str, str]]) -> Tuple[List[str], List[str], Dict[str, Tuple[str, str]]]:

    guess = guess_a_word(previous_guesses=guesses[:len(guesses)])
    guesses.append(guess)
    score = score_a_guess(secret_word, guess)
    styled_letters = [(letter, score, STYLES[score]) for letter, score in zip(guess, score)]
    # styled_guess = [f"{styled_letter_and_score(letter, score)}" for letter, score in zip(guess, score)]
    styled_guess = [f"[{style}]{letter}[/]" for letter, _, style in styled_letters]
    styled_guesses.append(" ".join(styled_guess))

    for letter, score, style in styled_letters:
        if SCORE_RANKING[score] > SCORE_RANKING[letter_status[letter][0]]:
            letter_status[letter] = (score, styled_letter_and_score(letter, score))

    return guesses, styled_guesses, letter_status

main()

──────────────────────────────── 🧙 Guess 1 💡  ────────────────────────────────

Guess word:

hjkew


H J K E W

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

...

A O G U S

D F G H J

E W H G K

A B C D E F G H I J K L M N O P Q R S T U V W X Y Z

Sorry, the word was ABODE